In [1]:
import numpy as np
import matplotlib.pyplot as plt
# from mpl_toolkits.mplot3d import Axes3D

from layers import DenseLayer
from cnn import NeuralNetwork

X = np.reshape([[0, 0], [0, 1], [1, 0], [1, 1]], (4, 2, 1))
Y = np.reshape([[0], [1], [1], [0]], (4, 1, 1))

layers = [
    DenseLayer(2, 3, activation="tanh"),
    DenseLayer(3, 1, activation="tanh"),
]

# train
model = NeuralNetwork()
model.layers = layers
model.train(X, Y, epochs=1000, learning_rate=0.1, batch_size=1)

# decision boundary plot
points = []
for x in np.linspace(0, 1, 20):
    for y in np.linspace(0, 1, 20):
        z = model.predict(np.reshape([[x], [y]], (2, 1)))
        points.append([x, y, z[0,0]])

points = np.array(points)

# fig = plt.figure()
# ax = fig.add_subplot(111, projection="3d")
# ax.scatter(points[:, 0], points[:, 1], points[:, 2], c=points[:, 2], cmap="winter")
# plt.show()

DenseLayer: 2 -> 3 (1, 2, 1)
DenseLayer: 3 -> 1 (1, 3, 1)
DenseLayer: 2 -> 3 (1, 2, 1)
DenseLayer: 3 -> 1 (1, 3, 1)
DenseLayer: 2 -> 3 (1, 2, 1)
DenseLayer: 3 -> 1 (1, 3, 1)
DenseLayer: 2 -> 3 (1, 2, 1)
DenseLayer: 3 -> 1 (1, 3, 1)
Epoch 0 Error 3.7160430407037626
DenseLayer: 2 -> 3 (1, 2, 1)
DenseLayer: 3 -> 1 (1, 3, 1)
DenseLayer: 2 -> 3 (1, 2, 1)
DenseLayer: 3 -> 1 (1, 3, 1)
DenseLayer: 2 -> 3 (1, 2, 1)
DenseLayer: 3 -> 1 (1, 3, 1)
DenseLayer: 2 -> 3 (1, 2, 1)
DenseLayer: 3 -> 1 (1, 3, 1)
Epoch 1 Error 1.189796911240597
DenseLayer: 2 -> 3 (1, 2, 1)
DenseLayer: 3 -> 1 (1, 3, 1)
DenseLayer: 2 -> 3 (1, 2, 1)
DenseLayer: 3 -> 1 (1, 3, 1)
DenseLayer: 2 -> 3 (1, 2, 1)
DenseLayer: 3 -> 1 (1, 3, 1)
DenseLayer: 2 -> 3 (1, 2, 1)
DenseLayer: 3 -> 1 (1, 3, 1)
Epoch 2 Error 1.0829846357198867
DenseLayer: 2 -> 3 (1, 2, 1)
DenseLayer: 3 -> 1 (1, 3, 1)
DenseLayer: 2 -> 3 (1, 2, 1)
DenseLayer: 3 -> 1 (1, 3, 1)
DenseLayer: 2 -> 3 (1, 2, 1)
DenseLayer: 3 -> 1 (1, 3, 1)
DenseLayer: 2 -> 3 (1, 2, 1)
Den

In [2]:
from keras.datasets import mnist
from keras import utils

In [3]:
%load_ext autoreload
%autoreload 2

import numpy as np

from layers import ConvLayer, FlattenLayer, DenseLayer, MaxPoolLayer, MeanPoolLayer
from losses import *
from cnn import NeuralNetwork

def preprocess_data(x, y, limit):
    zero_index = np.where(y == 0)[0][:limit]
    one_index = np.where(y == 1)[0][:limit]
    all_indices = np.hstack((zero_index, one_index))
    all_indices = np.random.permutation(all_indices)
    x, y = x[all_indices], y[all_indices]
    x = x.reshape(len(x), 1, 28, 28)
    x = x.astype("float32") / 255
    y = utils.to_categorical(y)
    y = y.reshape(len(y), 2, 1)
    return x, y

# load MNIST from server, limit to 100 images per class since we're not training on GPU
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, y_train = preprocess_data(x_train, y_train, 100)
x_test, y_test = preprocess_data(x_test, y_test, 100)

# neural network
layers = [
    ConvLayer(5, 3, (1, 28, 28), activation="sigmoid"),
    MeanPoolLayer((5, 26, 26), pool_size=(2,2)),
    ConvLayer(5, 5, (5, 13, 13), activation="sigmoid"),
    MaxPoolLayer((5, 9, 9), pool_size=(2,2)),
    FlattenLayer((5, 4, 4)),
    DenseLayer(80, 50, activation="sigmoid"),
    DenseLayer(50, 2, activation="sigmoid"),
]

model = NeuralNetwork(loss='binary_crossentropy')
model.layers = layers
# train
model.train(
    x_train,
    y_train,
    epochs=20,
    learning_rate=0.1,
    batch_size=1
)

# test
# for x, y in zip(x_test, y_test):
#     output = model.predict(x)
#     print(f"pred: {np.argmax(output)}, true: {np.argmax(y)}")

ConvLayer: (1, 28, 28) -> (5, 26, 26) (1, 1, 28, 28)
MeanPoolLayer: (5, 26, 26) -> (5, 13, 13) (1, 5, 26, 26)
ConvLayer: (5, 13, 13) -> (5, 9, 9) (1, 5, 13, 13)
MaxPoolLayer: (5, 9, 9) -> (5, 4, 4) (1, 5, 9, 9)
FlattenLayer: (5, 5, 5) -> (5, 25) (1, 5, 4, 4)
DenseLayer: 125 -> 50 (1, 80, 1)


ValueError: shapes (50,125) and (80,1) not aligned: 125 (dim 1) != 80 (dim 0)